In [3]:
######## SPAIN - SHP FILES

import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/La_Estrella_2020/Isochrones/Isochrones.shp')
print(gdf.crs)

gdf = gdf.to_crs(epsg=32719)
print(gdf.crs)


EPSG:25830
EPSG:32719


In [ ]:
import pandas as pd
import geopandas as gpd
import re

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/La_Estrella_2020/Isochrones/Isochrones.shp')
gdf = gdf.to_crs(epsg=32719)

def crear_iso_timestamp(row):
    # Extract date from 'fecha' and format 30072020 → 2020-07-30
    date_str = str(row['fecha'])
    if len(date_str) == 8:
        date_fmt = f"{date_str[4:8]}-{date_str[2:4]}-{date_str[0:2]}"
    else:
        date_fmt = ''
    # Extract hour from 'Descripcio'
    descripcio_str = str(row['Descripcio'])
    m = re.search(r'_(\d{4})$', descripcio_str)
    if m:
        hour_fmt = f"{m.group(1)[0:2]}:{m.group(1)[2:4]}"
    else:
        hour_fmt = ''
    # COmbine date and hour
    if date_fmt and hour_fmt:
        return f"{date_fmt} {hour_fmt}"
    else:
        return ''

# Create the new column
gdf['iso_timestamp'] = gdf.apply(crear_iso_timestamp, axis=1)

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Delete attribute
gdf = gdf.drop('Perim_m', axis=1)
gdf = gdf.drop('Superficie', axis=1)
gdf = gdf.drop('Id', axis=1)

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('LaEstrella2020Fire_test1.geojson', driver='GeoJSON')
